In [49]:
!pip install pandas altair altair_data_server swifter

  Using cached swifter-0.302-py3-none-any.whl (10 kB)
  Using cached numba-0.49.0-cp38-cp38-macosx_10_14_x86_64.whl (2.1 MB)
  Using cached tqdm-4.45.0-py2.py3-none-any.whl (60 kB)
  Using cached ipywidgets-7.5.1-py2.py3-none-any.whl (121 kB)
     |████████████████████████████████| 8.6 MB 8.7 MB/s eta 0:00:01
  Using cached llvmlite-0.32.0-cp38-cp38-macosx_10_9_x86_64.whl (15.9 MB)
  Using cached widgetsnbextension-3.5.1-py2.py3-none-any.whl (2.2 MB)
  Created wheel for bokeh: filename=bokeh-2.0.1-py3-none-any.whl size=9080029 sha256=08145f8b194e6227ed8e08a4f02903b20fe9a3cc4862703164a2baf72f4030ca
  Stored in directory: /Users/saul/Library/Caches/pip/wheels/03/5c/a3/49a9ae757ea6918039ffeba015622af9453f91d1f7a7b59be6
Successfully built bokeh


In [1]:
import pandas
import altair as alt
import pandas.io.json
import swifter
# import dask.dataframe
# from dask.distributed import Client

# https://github.com/pandas-dev/pandas/issues/26068#issuecomment-550900208
import json
pandas.io.json._json.loads = lambda s, *a, **kw: json.loads(s)

# dask.config.set({'temporary_directory': '/tmp'})
alt.data_transformers.enable('data_server')
pandas.set_option('max_colwidth', 200)

# client = Client()
# client

In [2]:
# data = dask.dataframe.read_json(
#     'skimage.jsonl',
#     blocksize=1000 * 1024,
#     meta=pandas.DataFrame.from_dict({"function": ["str"], "params": [{}]})
# )
# data
data = pandas.read_json('skimage.jsonl', lines=True)

In [3]:
data = data.set_index('function')
data.info()
data

<class 'pandas.core.frame.DataFrame'>
Index: 10293118 entries, builtins.getattr to builtins.getattr
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   params  object
dtypes: object(1)
memory usage: 157.1+ MB


,params
function,
builtins.getattr,"{'0': {'t': 'builtins.module', 'v': 'numpy'}, '1': 'byte'}"
builtins.getattr,"{'0': {'t': 'builtins.module', 'v': 'numpy'}, '1': 'ubyte'}"
builtins.getattr,"{'0': {'t': 'builtins.module', 'v': 'numpy'}, '1': 'short'}"
builtins.getattr,"{'0': {'t': 'builtins.module', 'v': 'numpy'}, '1': 'ushort'}"
builtins.getattr,"{'0': {'t': 'builtins.module', 'v': 'numpy'}, '1': 'intc'}"
...,...
builtins.getattr,"{'0': {'t': 'numpy.ndarray', 'v': [[4], 'uint8']}, '1': 'ndim'}"
numpy.arange,{'0': 8}
numpy.ndarray.reshape,"{'0': {'t': 'numpy.ndarray', 'v': [[8], 'int64']}, '1': {'t': 'tuple', 'v': [2, 2, 2]}}"


In [8]:
counts = data.groupby('function').count()
counts['occurance'] = counts['params'] / counts['params'].sum()

In [11]:
alt.Chart(counts.reset_index()).mark_bar().encode(
    y=alt.Y('function:N', sort='-x'),
    x=alt.X('occurance:Q', axis=alt.Axis(format='%'), scale=alt.Scale(type='log')),
    tooltip=['function', 'occurance', 'params']
)

alt.Chart(...)

In [ ]:
getitem = get_args('_operator.getitem')

/usr/local/Caskroom/miniconda/base/envs/scikit-image/lib/python3.8/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [13]:
def stringify(v):
    """
    Turn a complex type into a simpler string, yes with less data, but easier to group by.
    """
    if isinstance(v, int):
        return "int"
    if isinstance(v, str):
        return str(repr(v))
    if isinstance(v, float):
        return "float"
    if isinstance(v, list):
        return f"List[{stringify(v[0]) if v else 'Any'}]"
    if v is None:
        return 'None'
    t = v['t']
    if t == 'builtins.type':
        return f"Type[{v['v']}]"
    if t == 'tuple':
        return f"Tuple[{', '.join(stringify(a) for a in v['v'])}]"
    return t
        

In [14]:
def get_args(fn_name):
    return data[data.index == fn_name].swifter.apply(
        lambda x: pandas.Series({k: stringify(v) for k, v in x['params'].items()}),
        axis=1
    )

In [15]:
def make_arg_chart(df):
    selections = [alt.selection_multi(fields=[col]) for col in df.columns]
    charts = []
    for i, col in enumerate(df.columns):
        new_chart = alt.Chart(df).mark_bar().encode(
            alt.Y(col, sort='-x', type='nominal'),
            alt.X('count()', scale=alt.Scale(type='log')),
            color=alt.condition(selections[i], alt.value('lightblue'), alt.value('lightgray'))
        ).add_selection(
            selections[i]
        )
        for other_i, selection in enumerate(selections):
            # don't filter on my own selection
            if other_i == i:
                continue
            new_chart = new_chart.transform_filter(selection)
        charts.append(new_chart)
    return alt.concat(*charts, columns=3)

In [16]:
def inspect_functionn(fn_name):
    df = get_args(fn_name)
    return make_arg_chart(df)